In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [2]:
#DWPR01P
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="C#ntury123",
#     dsn="racorap33-scan.corp.intranet/dwpr01p_users",
#     encoding="UTF-8")

# CDW01P
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="C#ntury123",
#     dsn="RACORAP16-SCAN.IDC1.LEVEL3.COM/CDW01P_USERS",
#     encoding="UTF-8")

# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

# DASP001P
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="L2022$Um",
#     dsn="racorap27-scan.corp.intranet/dasp001p_prod",
#     encoding="UTF-8")

#ASL01P
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="C#ntury123",
#     dsn="RACORAP16-SCAN.IDC1.LEVEL3.COM/ASL01P_USERS",
#     encoding="UTF-8")

#DWODS02P - UNIQUAL
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="C3ntury123",
#     dsn="DWODS02Pdb.qintra.com:1523/DWODS02P",
#     encoding="UTF-8")

#CCDWO1P - SALES
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="C#ntury123",
#     dsn="racorap33-scan.corp.intranet:1521/ccdw01p_users",
#     encoding="UTF-8")

#CCDW02P - SALES

In [17]:
# qualifications

# select distinct DTN, WIRE_CENTER_ID
# from LQSMO.LU_CIRCUITS_O;


# 10/7/22 - update query to count unique LU where servicetype like XGS
# 10/19/22 - update details query to include NDSJOBID - per Scott request
query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            ), MG AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
                    -- add multigig polygons below --
                    AND UNI_POLYGONID IN ('1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1180028', '1178694', '1179569', '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1179808','1179976', -- MPLSMNGE
                                          '1179583' -- DNVRCODC
                                          )
                    
            ), QMG AS (
            SELECT  DISTINCT
                    QD.LUID
                    ,QD.STATE
                    ,QD.PRIMARY_WIRE_CENTER_ID
                    ,QD.ADDRESS
                    ,QD.UNI_SERVICETYPE
                    ,QD.OLT
                    ,QD.TECH_TYPE
                    ,QD.OLT_TYPE
                    ,QD.FSAI
                    ,QD.UNI_POLYGONID
                    ,QD.UNI_STATUS
                    ,QD.UNI_NDSJOBNUMBER
                    ,QD.UNI_FWRKSJOBNUMBER
                    ,MG.UNI_POLYGONID AS MG_ENABLED_POLYGON

            FROM    QD
                    LEFT JOIN MG
                        ON QD.UNI_POLYGONID = MG.UNI_POLYGONID            
            )
            
            SELECT  0 as s
                    ,STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,UNI_SERVICETYPE
                    ,count(distinct LUID) as cnt
                    ,case when UNI_SERVICETYPE like '%XGS%' then count(distinct LUID) end as servicetype_xgs_cnt
                    ,count(case when MG_ENABLED_POLYGON is not null then 1 end) as multigig_cnt
                    
            FROM    QMG
                    
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,UNI_SERVICETYPE

            UNION ALL
            
            SELECT  1
                    ,'ALL'
                    ,'GRAND TOTAL'
                    ,'ALL'
                    ,'ALL'
                    ,count(distinct LUID)
                    ,count(distinct case when UNI_SERVICETYPE like '%XGS%' then LUID end)
                    ,count(case when MG_ENABLED_POLYGON is not null then 1 end)
                    
            FROM    QMG
            
            ORDER BY 
                    s
                    ,STATE

                                """

query2 = """
            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_POLYGONID
                    ,O.UNI_SERVICETYPE
                    ,O.UNI_STATUS
                    ,O.TECH_TYPE
                    ,O.FSAI
                    ,O.OLT_TYPE
                    ,O.ADDRESS
                    ,O.OLT
                    ,O.UNI_ECD
                    ,O.NDSJOBID
                    --O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  *
                    
            FROM    QD

                                """

# query3 = """

# select *
# FROM    LQSMO.LU_QUAL_O O

# """

df_ora = pd.read_sql(query, con=conn)
df_ora_details = pd.read_sql(query2, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [18]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df_ora.rename(columns = {"CNT": "LU_QUAL_O_CNT",
                         "UNI_POLYGONID": "POLYGON_ID"}, inplace = True)
df_ora.sort_values(by = ['S','STATE']).head() # contains state/wc/polygon info

,S,STATE,PRIMARY_WIRE_CENTER_ID,POLYGON_ID,UNI_SERVICETYPE,LU_QUAL_O_CNT,SERVICETYPE_XGS_CNT,MULTIGIG_CNT
0,0,AZ,TCSNAZMA,1179803,XGS,332,332.0,0
1,0,AZ,TCSNAZCR,1179641,XGS,176,176.0,0
2,0,AZ,TCSNAZMA,1179704,XGS,545,545.0,0
3,0,AZ,TCSNAZMA,1179701,XGS,367,367.0,0
4,0,AZ,TCSNAZMA,1179793,XGS,133,133.0,0


In [53]:
# check data
df_ora.loc[df_ora["MULTIGIG_CNT"]>0,:]

,S,STATE,PRIMARY_WIRE_CENTER_ID,POLYGON_ID,LU_QUAL_O_CNT,SERVICETYPE_XGS_CNT,MULTIGIG_CNT
0,0,AZ,PHNXAZNE,1179499,212,212.0,212
1,0,AZ,PHNXAZSO,1179121,247,247.0,247
2,0,AZ,PHNXAZSO,1179498,319,319.0,319
3,0,AZ,PHNXAZSO,1179547,284,284.0,284
36,0,CO,ARVDCOMA,1178832,196,196.0,196
61,0,CO,DNVRCODC,1179583,166,166.0,166
62,0,CO,DNVRCOMA,1179864,437,437.0,437
64,0,CO,GLDNCOMA,1179674,158,158.0,158
65,0,CO,GLDNCOMA,1179687,183,183.0,183
66,0,CO,GLDNCOMA,1180254,71,71.0,71


In [54]:
ST_df = pd.read_excel(r'Qualifications ST/Data.xlsx',sheet_name = 'Export')

ST_Details_df = ST_df.copy() # create copy to export

ST_df["Polygon ID - Phase Else Site"] = ST_df["Polygon ID - Phase Else Site"].fillna("None")
ST_df = ST_df.groupby(by = ["State","PRIMARY_WIRE_CENTER_ID","Polygon ID - Phase Else Site"], as_index = False)["# of LUs - Phase Else Site"].sum()
ST_df = ST_df.rename(columns = {'State': 'STATE',
                                'Polygon ID - Phase Else Site': 'POLYGON_ID'})
#ST_df

C:\Users\AD21236\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [56]:
# polygon level info starts here

df_ora_state = df_ora.copy() # use this for state summary with unitype cnt
df_ora = df_ora.groupby(['S','STATE','PRIMARY_WIRE_CENTER_ID','POLYGON_ID'], as_index = False).sum()
Q_df = ST_df.merge(df_ora[["STATE","PRIMARY_WIRE_CENTER_ID", "POLYGON_ID","LU_QUAL_O_CNT","MULTIGIG_CNT"]], how = "outer", on = ["PRIMARY_WIRE_CENTER_ID","STATE","POLYGON_ID"]).fillna(0)
Q_df[["# of LUs - Phase Else Site", "LU_QUAL_O_CNT"]] = Q_df[["# of LUs - Phase Else Site", "LU_QUAL_O_CNT"]].astype(int)
Q_df.loc[Q_df["PRIMARY_WIRE_CENTER_ID"] == "GRAND TOTAL", ["# of LUs - Phase Else Site"]] = Q_df["# of LUs - Phase Else Site"].sum()
Q_df = Q_df[['STATE','PRIMARY_WIRE_CENTER_ID','POLYGON_ID','# of LUs - Phase Else Site','LU_QUAL_O_CNT','MULTIGIG_CNT']]
Q_poly_df = Q_df.copy() # polygon level summary ready for export

In [60]:
# wc level information start here
Q_df = Q_df.groupby(by = ['STATE','PRIMARY_WIRE_CENTER_ID'], as_index = False).sum() # wc level info ready for export
Q_df['ORDER'] = np.where(Q_df['PRIMARY_WIRE_CENTER_ID'] == 'GRAND TOTAL', 1, 0)
Q_df = Q_df.sort_values(by = 'ORDER')
Q_df = Q_df.drop(['ORDER'], axis = 1)

In [62]:
# state summary begins here

df_ora_state = df_ora_state.fillna(0)
Q_State = df_ora_state.groupby(['STATE'], as_index = False).sum()
Q_State['STATE'] = Q_State['STATE'].replace(['ALL'], 'GRAND TOTAL')
Q_State

,STATE,S,LU_QUAL_O_CNT,SERVICETYPE_XGS_CNT,MULTIGIG_CNT
0,GRAND TOTAL,1,49964,49025.0,8431
1,AZ,0,10722,10498.0,1062
2,CO,0,7806,7806.0,1211
3,FL,0,19174,18459.0,0
4,MN,0,7474,7474.0,5742
5,OR,0,264,264.0,74
6,UT,0,2199,2199.0,0
7,WA,0,2325,2325.0,342


In [63]:
# join lu qual to sitetracker at state level
Q_df_st = Q_df[['STATE','# of LUs - Phase Else Site']].groupby('STATE', as_index = False).sum()
Q_df_st.loc[Q_df_st['STATE'] == 'ALL','STATE'] = 'GRAND TOTAL'
Q_State = Q_State.merge(Q_df_st, how = 'outer', on = 'STATE')
Q_State = Q_State.loc[(Q_State['# of LUs - Phase Else Site']>0) | (Q_State['LU_QUAL_O_CNT']>0),['STATE','# of LUs - Phase Else Site','LU_QUAL_O_CNT','SERVICETYPE_XGS_CNT']]

Q_State['ORDER'] = np.where(Q_State['STATE'] == 'GRAND TOTAL', 1, 0)
Q_State = Q_State.sort_values(by = 'ORDER')
Q_State = Q_State.drop(['ORDER'], axis = 1)
Q_State = Q_State.fillna(0)
Q_State

,STATE,# of LUs - Phase Else Site,LU_QUAL_O_CNT,SERVICETYPE_XGS_CNT
1,AZ,11648,10722.0,10498.0
2,CO,8324,7806.0,7806.0
3,FL,19799,19174.0,18459.0
4,MN,7474,7474.0,7474.0
5,OR,254,264.0,264.0
6,UT,2554,2199.0,2199.0
7,WA,2237,2325.0,2325.0
8,ID,102,0.0,0.0
0,GRAND TOTAL,52392,49964.0,49025.0


In [65]:
import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_LUQ_SITETRACKER_Summary.xlsx"


with pd.ExcelWriter(excelfilename) as writer:
    Q_poly_df.to_excel(writer, sheet_name='PolygonID_Summary', index = False)
    Q_df.to_excel(writer, sheet_name = "WC_Summary", index=False)
    Q_State.to_excel(writer, sheet_name='State_Summary', index = False)
    ST_Details_df.to_excel(writer, sheet_name='ST_Details', index = False)
    df_ora_details.to_excel(writer, sheet_name='LUQUAL_Details', index = False)
    

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()

In [27]:
conn.close()